In [1]:
import os
import json
from typing import Dict, List, Optional, Any, Callable, Tuple
from abc import ABC, abstractmethod
from datetime import datetime
import openai
from dataclasses import dataclass, field
from enum import Enum
import re


class HandoffMode(Enum):
    """Defines how control should be transferred between agents"""
    DIRECT_TO_USER = "direct_to_user"  # Sub-agent interacts directly with user
    THROUGH_MAIN = "through_main"      # All interactions go through main agent


class ConversationState(Enum):
    """Tracks the current state of the conversation"""
    IDLE = "idle"
    ACTIVE_CONVERSATION = "active_conversation"
    HANDOFF_PENDING = "handoff_pending"
    MULTI_AGENT_COORDINATION = "multi_agent_coordination"


@dataclass
class Message:
    """Represents a message in the conversation"""
    role: str  # 'user', 'assistant', 'system'
    content: str
    agent_id: str
    timestamp: datetime = field(default_factory=datetime.now)
    metadata: Dict[str, Any] = field(default_factory=dict)
    requires_handoff_check: bool = False


@dataclass
class HandoffRequest:
    """Represents a handoff request from one agent to another"""
    from_agent: str
    to_agent: str
    reason: str
    context: List[Message]
    handoff_mode: HandoffMode = HandoffMode.DIRECT_TO_USER
    priority: str = "normal"  # 'urgent', 'normal', 'low'
    metadata: Dict[str, Any] = field(default_factory=dict)


@dataclass
class AgentCapability:
    """Describes what an agent can do"""
    name: str
    description: str
    keywords: List[str]
    examples: List[str]
    domain_patterns: List[str] = field(default_factory=list)  # Regex patterns for domain detection


@dataclass
class ConversationContext:
    """Maintains the full context of a conversation"""
    messages: List[Message]
    current_topic: str
    active_agents: List[str]
    handoff_count: int = 0
    session_id: str = field(default_factory=lambda: datetime.now().strftime("%Y%m%d_%H%M%S"))


class BaseAgent(ABC):
    """Abstract base class for all agents"""
    
    def __init__(self, agent_id: str, name: str, description: str):
        self.agent_id = agent_id
        self.name = name
        self.description = description
        self.conversation_history: List[Message] = []
        self.active_context: Optional[ConversationContext] = None
        
    @abstractmethod
    def get_capabilities(self) -> AgentCapability:
        """Return the capabilities of this agent"""
        pass
    
    @abstractmethod
    def process_message(self, message: str, context: List[Message]) -> Tuple[str, Optional[HandoffRequest]]:
        """
        Process a message and return response and optional handoff request
        
        Returns:
            tuple: (response_text, optional_handoff_request)
        """
        pass
    
    def should_handle_query(self, message: str, context: List[Message]) -> Tuple[bool, float]:
        """
        Determine if this agent should handle the query
        
        Returns:
            tuple: (should_handle, confidence_score)
        """
        capability = self.get_capabilities()
        
        # Check keywords
        message_lower = message.lower()
        keyword_matches = sum(1 for keyword in capability.keywords if keyword in message_lower)
        
        # Check domain patterns
        pattern_matches = 0
        for pattern in capability.domain_patterns:
            if re.search(pattern, message, re.IGNORECASE):
                pattern_matches += 1
        
        # Calculate confidence score
        confidence = (keyword_matches * 0.3 + pattern_matches * 0.7)
        
        # Check context relevance
        if context and len(context) > 0:
            recent_agent_messages = [m for m in context[-5:] if m.agent_id == self.agent_id]
            if recent_agent_messages:
                confidence += 0.3  # Boost if recently active
        
        return confidence > 0.3, min(confidence, 1.0)
    
    def add_to_history(self, message: Message):
        """Add a message to conversation history"""
        self.conversation_history.append(message)
        
    def get_conversation_summary(self) -> str:
        """Get a summary of the conversation for this agent"""
        if not self.conversation_history:
            return "No conversation history"
        
        summary = f"Conversation summary for {self.name}:\n"
        summary += f"Total messages: {len(self.conversation_history)}\n"
        summary += f"Last interaction: {self.conversation_history[-1].timestamp}\n"
        return summary


class MainOrchestrator:
    """Main orchestrator that manages all agents and handoffs"""
    
    def __init__(self, openai_api_key: str, model: str = "gpt-4o"):
        self.openai_client = openai.OpenAI(api_key=openai_api_key)
        self.model = model
        self.agents: Dict[str, BaseAgent] = {}
        self.current_agent_id: Optional[str] = None
        self.conversation_context: ConversationContext = ConversationContext(
            messages=[],
            current_topic="general",
            active_agents=[]
        )
        self.handoff_history: List[HandoffRequest] = []
        self.agent_memory: Dict[str, List[Dict[str, Any]]] = {}
        self.conversation_state = ConversationState.IDLE
        
    def register_agent(self, agent: BaseAgent):
        """Register a new agent with the orchestrator"""
        self.agents[agent.agent_id] = agent
        self.agent_memory[agent.agent_id] = []
        print(f"✅ Registered agent: {agent.name} ({agent.agent_id})")
        
    def _analyze_handoff_need(self, user_message: str, current_response: str) -> Dict[str, Any]:
        """Analyze if handoff is needed based on conversation flow"""
        prompt = f"""Analyze this conversation and determine if a handoff to a different agent is needed.

Current Agent: {self.current_agent_id}
User Message: "{user_message}"
Agent Response: "{current_response}"

Available Agents:
{self._get_agents_description()}

Recent Context:
{self._get_recent_context(3)}

Determine:
1. Is the current agent still the best choice?
2. Is the user asking about something outside current agent's domain?
3. Would another agent provide better assistance?

Respond in JSON:
{{
    "needs_handoff": true/false,
    "suggested_agent": "agent_id or null",
    "reason": "explanation",
    "confidence": 0.0-1.0
}}"""

        try:
            response = self.openai_client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You analyze conversations to determine optimal agent routing."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                response_format={"type": "json_object"}
            )
            
            return json.loads(response.choices[0].message.content)
        except Exception as e:
            print(f"Error in handoff analysis: {e}")
            return {"needs_handoff": False, "confidence": 0.0}
    
    def _get_agents_description(self) -> str:
        """Get formatted description of all agents"""
        descriptions = []
        for agent_id, agent in self.agents.items():
            cap = agent.get_capabilities()
            descriptions.append(f"- {agent_id}: {agent.description} (Keywords: {', '.join(cap.keywords[:5])})")
        return "\n".join(descriptions)
    
    def _build_agent_selection_prompt(self, user_message: str) -> str:
        """Build prompt for selecting appropriate agent"""
        agent_descriptions = []
        
        # Get confidence scores from each agent
        agent_scores = {}
        for agent_id, agent in self.agents.items():
            should_handle, confidence = agent.should_handle_query(
                user_message, 
                self.conversation_context.messages
            )
            agent_scores[agent_id] = confidence
            
            capability = agent.get_capabilities()
            agent_descriptions.append(f"""
Agent ID: {agent_id}
Name: {agent.name}
Description: {agent.description}
Confidence Score: {confidence:.2f}
Keywords: {', '.join(capability.keywords)}
Domain Patterns: {len(capability.domain_patterns)} patterns defined
Recent Activity: {len([m for m in self.conversation_context.messages[-10:] if m.agent_id == agent_id])} messages
""")
        
        recent_context = self._get_recent_context(5)
        
        prompt = f"""You are an intelligent orchestrator that routes user queries to appropriate agents.

Current Conversation State: {self.conversation_state.value}
Current Agent: {self.current_agent_id or 'None'}
Handoff Count in Session: {self.conversation_context.handoff_count}

Available agents with confidence scores:
{chr(10).join(agent_descriptions)}

Recent conversation context:
{recent_context}

User message: "{user_message}"

Based on the user's message, conversation context, and agent confidence scores, determine:
1. Which agent should handle this query
2. Whether to handoff control or continue with current agent
3. The handoff mode and priority

Rules:
- Prefer the current agent if confidence difference is < 0.3
- Consider user's explicit requests for different agents
- Minimize unnecessary handoffs
- Use DIRECT_TO_USER for domain-specific extended conversations
- Use THROUGH_MAIN when coordination between agents might be needed

Respond in JSON format:
{{
    "selected_agent_id": "agent_id",
    "reason": "detailed explanation",
    "handoff_mode": "direct_to_user" or "through_main",
    "requires_handoff": true/false,
    "confidence": 0.0-1.0,
    "priority": "urgent/normal/low"
}}"""
        return prompt
    
    def _get_recent_context(self, n: int = 5) -> str:
        """Get recent conversation context"""
        recent_messages = self.conversation_context.messages[-n:] if len(self.conversation_context.messages) > n else self.conversation_context.messages
        context_str = ""
        for msg in recent_messages:
            agent_name = self.agents[msg.agent_id].name if msg.agent_id in self.agents else msg.agent_id
            context_str += f"{msg.role} ({agent_name}): {msg.content[:100]}...\n"
        return context_str or "No previous context"
    
    def _select_agent(self, user_message: str) -> Dict[str, Any]:
        """Use LLM to select appropriate agent with agent confidence scores"""
        prompt = self._build_agent_selection_prompt(user_message)
        
        try:
            response = self.openai_client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are an intelligent agent router. Always respond with valid JSON."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            return result
            
        except Exception as e:
            print(f"Error in agent selection: {e}")
            # Fallback to agent with highest confidence
            if self.agents:
                best_agent = max(
                    self.agents.items(),
                    key=lambda x: x[1].should_handle_query(user_message, self.conversation_context.messages)[1]
                )
                return {
                    "selected_agent_id": best_agent[0],
                    "reason": "Selected based on highest confidence score",
                    "handoff_mode": "direct_to_user",
                    "requires_handoff": best_agent[0] != self.current_agent_id,
                    "confidence": 0.7,
                    "priority": "normal"
                }
            return None
    
    def _update_agent_memory(self, agent_id: str, interaction: Dict[str, Any]):
        """Update memory about agent interactions"""
        self.agent_memory[agent_id].append({
            "timestamp": datetime.now().isoformat(),
            "interaction": interaction,
            "session_id": self.conversation_context.session_id
        })
        
        # Keep only last 100 interactions per agent
        if len(self.agent_memory[agent_id]) > 100:
            self.agent_memory[agent_id] = self.agent_memory[agent_id][-100:]
    
    def _perform_handoff(self, from_agent: str, to_agent: str, reason: str, priority: str = "normal"):
        """Perform a handoff between agents"""
        handoff = HandoffRequest(
            from_agent=from_agent,
            to_agent=to_agent,
            reason=reason,
            context=self.conversation_context.messages[-10:],
            handoff_mode=HandoffMode.DIRECT_TO_USER,
            priority=priority
        )
        
        self.handoff_history.append(handoff)
        self.conversation_context.handoff_count += 1
        self.current_agent_id = to_agent
        
        # Update active agents
        if to_agent not in self.conversation_context.active_agents:
            self.conversation_context.active_agents.append(to_agent)
        
        # Create handoff notification
        handoff_msg = f"🔄 Transferring to {self.agents[to_agent].name}"
        if priority == "urgent":
            handoff_msg = f"🚨 {handoff_msg} (Priority: Urgent)"
        
        return f"{handoff_msg}\n💬 {reason}"
    
    def process_user_message(self, user_message: str) -> str:
        """Process a user message and return response"""
        # Update conversation state
        self.conversation_state = ConversationState.ACTIVE_CONVERSATION
        
        # Add user message to context
        user_msg = Message(
            role="user",
            content=user_message,
            agent_id="user",
            requires_handoff_check=True
        )
        self.conversation_context.messages.append(user_msg)
        
        # Select appropriate agent
        selection = self._select_agent(user_message)
        
        if not selection or selection["selected_agent_id"] not in self.agents:
            return "I'm sorry, but I couldn't find an appropriate agent to handle your request."
        
        selected_agent_id = selection["selected_agent_id"]
        
        # Check if we need to handoff
        handoff_message = ""
        if selection["requires_handoff"] and selected_agent_id != self.current_agent_id:
            handoff_message = self._perform_handoff(
                self.current_agent_id or "main",
                selected_agent_id,
                selection["reason"],
                selection.get("priority", "normal")
            )
            handoff_message += "\n\n"
        
        # Set current agent if not set
        if not self.current_agent_id:
            self.current_agent_id = selected_agent_id
            if selected_agent_id not in self.conversation_context.active_agents:
                self.conversation_context.active_agents.append(selected_agent_id)
        
        # Process message with selected agent
        agent = self.agents[self.current_agent_id]
        response, handoff_request = agent.process_message(
            user_message, 
            self.conversation_context.messages
        )
        
        # Add agent response to context
        agent_msg = Message(
            role="assistant",
            content=response,
            agent_id=self.current_agent_id,
            metadata={"confidence": selection.get("confidence", 0.8)}
        )
        self.conversation_context.messages.append(agent_msg)
        agent.add_to_history(agent_msg)
        
        # Check for implicit handoff needs
        if not handoff_request and len(response) > 50:
            handoff_analysis = self._analyze_handoff_need(user_message, response)
            if handoff_analysis.get("needs_handoff") and handoff_analysis.get("confidence", 0) > 0.7:
                handoff_request = HandoffRequest(
                    from_agent=self.current_agent_id,
                    to_agent=handoff_analysis["suggested_agent"],
                    reason=handoff_analysis["reason"],
                    context=self.conversation_context.messages[-5:],
                    handoff_mode=HandoffMode.DIRECT_TO_USER
                )
        
        # Update agent memory
        self._update_agent_memory(self.current_agent_id, {
            "user_message": user_message,
            "response": response,
            "handoff_requested": handoff_request is not None,
            "selection_confidence": selection.get("confidence", 0.8)
        })
        
        # Handle handoff request from agent
        additional_message = ""
        if handoff_request and handoff_request.to_agent in self.agents:
            additional_message = "\n\n" + self._perform_handoff(
                handoff_request.from_agent,
                handoff_request.to_agent,
                handoff_request.reason,
                handoff_request.priority
            )
        
        return handoff_message + response + additional_message
    
    def get_conversation_summary(self) -> Dict[str, Any]:
        """Get detailed summary of current conversation state"""
        agent_participation = {}
        for msg in self.conversation_context.messages:
            if msg.agent_id in self.agents:
                agent_participation[msg.agent_id] = agent_participation.get(msg.agent_id, 0) + 1
        
        return {
            "session_id": self.conversation_context.session_id,
            "current_agent": self.current_agent_id,
            "conversation_state": self.conversation_state.value,
            "total_messages": len(self.conversation_context.messages),
            "total_handoffs": self.conversation_context.handoff_count,
            "active_agents": self.conversation_context.active_agents,
            "agent_participation": agent_participation,
            "recent_handoffs": [
                {
                    "from": h.from_agent,
                    "to": h.to_agent,
                    "reason": h.reason,
                    "priority": h.priority
                }
                for h in self.handoff_history[-5:]
            ],
            "current_topic": self.conversation_context.current_topic
        }
    
    def reset_conversation(self):
        """Reset the conversation context"""
        self.conversation_context = ConversationContext(
            messages=[],
            current_topic="general",
            active_agents=[]
        )
        self.current_agent_id = None
        self.conversation_state = ConversationState.IDLE
        self.handoff_history = []


# Example implementation of specialized agents
class CustomerSupportAgent(BaseAgent):
    """Customer support agent with enhanced capabilities"""
    
    def __init__(self, openai_api_key: str):
        super().__init__(
            agent_id="customer_support",
            name="Customer Support Agent",
            description="Handles customer inquiries, complaints, and support requests"
        )
        self.openai_client = openai.OpenAI(api_key=openai_api_key)
        
    def get_capabilities(self) -> AgentCapability:
        return AgentCapability(
            name=self.name,
            description=self.description,
            keywords=["support", "help", "issue", "problem", "complaint", "refund", "return", "order", "shipping"],
            examples=[
                "I need help with my order",
                "How do I return this item?",
                "I have a complaint about the service",
                "Where is my package?"
            ],
            domain_patterns=[
                r"(help|support|assist)",
                r"(order|purchase|buy)",
                r"(return|refund|exchange)",
                r"(complaint|issue|problem)"
            ]
        )
    
    def process_message(self, message: str, context: List[Message]) -> Tuple[str, Optional[HandoffRequest]]:
        try:
            # Build context-aware prompt
            recent_context = "\n".join([f"{m.role}: {m.content}" for m in context[-3:]])
            
            response = self.openai_client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": f"""You are a helpful customer support agent. 
Be empathetic and solution-oriented. You have access to order systems and can help with:
- Order tracking and shipping inquiries
- Returns and refunds
- Product issues and complaints
- General customer service

Recent context:
{recent_context}

If the customer asks about technical product issues or bugs, suggest transferring to technical support."""},
                    {"role": "user", "content": message}
                ],
                temperature=0.7
            )
            
            response_text = response.choices[0].message.content
            
            # Check for handoff triggers
            handoff = None
            if any(word in message.lower() for word in ["technical", "bug", "error", "api", "integration"]):
                if "technical support" in response_text.lower() or "technical" in message.lower():
                    handoff = HandoffRequest(
                        from_agent=self.agent_id,
                        to_agent="technical_support",
                        reason="Customer needs technical assistance with product functionality",
                        context=context,
                        handoff_mode=HandoffMode.DIRECT_TO_USER,
                        priority="normal"
                    )
            elif any(word in message.lower() for word in ["sales", "pricing", "upgrade", "plan"]):
                handoff = HandoffRequest(
                    from_agent=self.agent_id,
                    to_agent="sales",
                    reason="Customer has questions about pricing or upgrades",
                    context=context,
                    handoff_mode=HandoffMode.DIRECT_TO_USER,
                    priority="normal"
                )
            
            return response_text, handoff
            
        except Exception as e:
            return f"I apologize for the inconvenience. Let me help you with that. Error: {str(e)}", None


class TechnicalSupportAgent(BaseAgent):
    """Technical support agent with enhanced capabilities"""
    
    def __init__(self, openai_api_key: str):
        super().__init__(
            agent_id="technical_support",
            name="Technical Support Agent",
            description="Handles technical issues, debugging, and technical guidance"
        )
        self.openai_client = openai.OpenAI(api_key=openai_api_key)
        
    def get_capabilities(self) -> AgentCapability:
        return AgentCapability(
            name=self.name,
            description=self.description,
            keywords=["technical", "bug", "error", "code", "api", "integration", "debug", "feature", "configuration"],
            examples=[
                "I'm getting an error in the API",
                "How do I integrate this feature?",
                "The system is showing a bug",
                "I need help with configuration"
            ],
            domain_patterns=[
                r"(error|bug|issue).*?(code|api|system)",
                r"(integrate|integration|api)",
                r"(debug|troubleshoot|fix)",
                r"(technical|tech).*?(support|help|issue)"
            ]
        )
    
    def process_message(self, message: str, context: List[Message]) -> Tuple[str, Optional[HandoffRequest]]:
        try:
            recent_context = "\n".join([f"{m.role}: {m.content}" for m in context[-3:]])
            
            response = self.openai_client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": f"""You are a technical support specialist. 
Provide clear, technical solutions. You can:
- Debug technical issues
- Help with API integration
- Troubleshoot errors and bugs
- Provide configuration guidance
- Explain technical features

Recent context:
{recent_context}

If the customer needs billing support or general customer service, suggest transferring to customer support."""},
                    {"role": "user", "content": message}
                ],
                temperature=0.5
            )
            
            response_text = response.choices[0].message.content
            
            # Check for handoff triggers
            handoff = None
            if any(word in message.lower() for word in ["refund", "payment", "billing", "invoice"]):
                handoff = HandoffRequest(
                    from_agent=self.agent_id,
                    to_agent="customer_support",
                    reason="Customer needs help with billing or payment issues",
                    context=context,
                    handoff_mode=HandoffMode.DIRECT_TO_USER,
                    priority="normal"
                )
            
            return response_text, handoff
            
        except Exception as e:
            return f"Technical error encountered: {str(e)}", None


class SalesAgent(BaseAgent):
    """Sales agent for handling sales inquiries"""
    
    def __init__(self, openai_api_key: str):
        super().__init__(
            agent_id="sales",
            name="Sales Agent",
            description="Handles sales inquiries, pricing, and product recommendations"
        )
        self.openai_client = openai.OpenAI(api_key=openai_api_key)
        
    def get_capabilities(self) -> AgentCapability:
        return AgentCapability(
            name=self.name,
            description=self.description,
            keywords=["sales", "pricing", "cost", "plan", "upgrade", "subscription", "demo", "trial"],
            examples=[
                "What are your pricing plans?",
                "I want to upgrade my subscription",
                "Can I get a demo?",
                "What's included in the enterprise plan?"
            ],
            domain_patterns=[
                r"(price|pricing|cost)",
                r"(plan|subscription|upgrade)",
                r"(demo|trial)",
                r"(sales|purchase|buy)"
            ]
        )
    
    def process_message(self, message: str, context: List[Message]) -> Tuple[str, Optional[HandoffRequest]]:
        try:
            response = self.openai_client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a knowledgeable sales agent. Help customers with pricing, plans, and product information. Be consultative and helpful."},
                    {"role": "user", "content": message}
                ],
                temperature=0.7
            )
            
            return response.choices[0].message.content, None
            
        except Exception as e:
            return f"I'd be happy to help with your sales inquiry. Error: {str(e)}", None


# Usage example with enhanced features
if __name__ == "__main__":
    # Initialize the orchestrator
    api_key = os.getenv("OPENAI_API_KEY", "your-api-key-here")
    orchestrator = MainOrchestrator(openai_api_key=api_key)
    
    # Register agents
    customer_agent = CustomerSupportAgent(api_key)
    technical_agent = TechnicalSupportAgent(api_key)
    sales_agent = SalesAgent(api_key)
    
    orchestrator.register_agent(customer_agent)
    orchestrator.register_agent(technical_agent)
    orchestrator.register_agent(sales_agent)
    
    # Example conversation
    print("🤖 Multi-Agent System Ready!")
    print("📝 Type 'quit' to exit, 'summary' for conversation summary, 'reset' to start new conversation\n")
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == 'quit':
            break
        elif user_input.lower() == 'summary':
            summary = orchestrator.get_conversation_summary()
            print("\n📊 Conversation Summary:")
            print(json.dumps(summary, indent=2))
            print()
            continue
        elif user_input.lower() == 'reset':
            orchestrator.reset_conversation()
            print("🔄 Conversation reset. Starting fresh!\n")
            continue
            
        response = orchestrator.process_user_message(user_input)
        print(f"\n{response}\n")
        
        # Show current state in a clean format
        summary = orchestrator.get_conversation_summary()
        status = f"[Agent: {orchestrator.agents[summary['current_agent']].name if summary['current_agent'] else 'None'}"
        status += f" | Handoffs: {summary['total_handoffs']}"
        status += f" | Active Agents: {len(summary['active_agents'])}]"
        print(f"{status}\n")

✅ Registered agent: Customer Support Agent (customer_support)
✅ Registered agent: Technical Support Agent (technical_support)
✅ Registered agent: Sales Agent (sales)
🤖 Multi-Agent System Ready!
📝 Type 'quit' to exit, 'summary' for conversation summary, 'reset' to start new conversation



You:  hello I am want tech support on python 



🔄 Transferring to Technical Support Agent
💬 The user's message explicitly requests technical support related to Python, which aligns with the expertise of the Technical Support Agent. Despite the confidence score being lower than Customer Support, the request is clearly technical in nature.

Hello! I'd be happy to help you with Python. Could you please provide more details about the issue or the specific area you need assistance with? This could be related to debugging, API integration, errors, configuration, or understanding certain Python features. Let me know how I can assist you!

[Agent: Technical Support Agent | Handoffs: 1 | Active Agents: 1]



You:  what is decorator 



In Python, a decorator is a design pattern that allows you to modify or extend the behavior of functions or methods without permanently modifying their source code. Decorators are typically used to add functionality to an existing object in a clean and readable way.

A decorator is a function that takes another function as an argument and returns a new function with added or altered behavior. Decorators are often used for logging, access control, instrumentation, and caching, among other things.

Here's a simple example of a decorator:

```python
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()
```

In this example:
- `my_decorator` is a decorator function that takes `say_hello` as an argument.
- The `wrapper` function adds behavior before and after calli

...

You:  what is class 



In Java, a class is a blueprint or template for creating objects. It defines a set of properties (fields) and behaviors (methods) that the objects created from the class will have. Classes are fundamental to object-oriented programming, as they encapsulate data and functionality together.

Here's a basic structure of a class in Java:

```java
public class Car {
    // Fields (attributes)
    private String color;
    private String model;
    private int year;

    // Constructor
    public Car(String color, String model, int year) {
        this.color = color;
        this.model = model;
        this.year = year;
    }

    // Method (behavior)
    public void displayInfo() {
        System.out.println("Car model: " + model + ", Color: " + color + ", Year: " + year);
    }

    // Getter for the color field
    public String getColor() {
        return color;
    }

    // Setter for the color field
    public void setColor(String color) {
        this.color = color;
    }
}
```

###

You:  what did I ask about in python ?



It seems like you were asking about the concept of a "class" in programming. In Python, a class is similar to Java in that it serves as a blueprint for creating objects. It defines properties (attributes) and behaviors (methods) that the objects created from the class will have.

Here's a basic structure of a class in Python:

```python
class Car:
    # Constructor
    def __init__(self, color, model, year):
        # Attributes
        self.color = color
        self.model = model
        self.year = year

    # Method
    def display_info(self):
        print(f"Car model: {self.model}, Color: {self.color}, Year: {self.year}")

    # Getter for the color attribute
    def get_color(self):
        return self.color

    # Setter for the color attribute
    def set_color(self, color):
        self.color = color

# Using the class
my_car = Car("Red", "Toyota", 2020)
my_car.display_info()

# Accessing and modifying attributes using getter and setter
print("Original color:", my_car.get_co

You:  quit


In [3]:
"""
Multi-Agent Orchestration System with Handoff Capabilities
A flexible system for managing multiple agents with intelligent routing and context preservation
"""

import json
import logging
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum
from typing import Any, Dict, List, Optional, Tuple, Callable
from collections import defaultdict
import openai
import requests
from pydantic import BaseModel

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class HandoffDecision(Enum):
    """Decisions for agent handoff"""
    CONTINUE = "continue"  # Current agent continues
    HANDOFF = "handoff"   # Transfer to another agent
    RETURN_TO_MAIN = "return_to_main"  # Return control to orchestrator


@dataclass
class Message:
    """Represents a message in the conversation"""
    role: str  # 'user', 'assistant', 'system'
    content: str
    timestamp: datetime = field(default_factory=datetime.now)
    agent_id: Optional[str] = None
    metadata: Dict[str, Any] = field(default_factory=dict)


@dataclass
class ConversationContext:
    """Maintains conversation context and history"""
    messages: List[Message] = field(default_factory=list)
    current_agent: Optional[str] = None
    previous_agents: List[str] = field(default_factory=list)
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def add_message(self, message: Message):
        """Add a message to the conversation history"""
        self.messages.append(message)
    
    def get_recent_messages(self, n: int = 10) -> List[Message]:
        """Get the n most recent messages"""
        return self.messages[-n:]
    
    def get_context_summary(self) -> str:
        """Generate a summary of the conversation context"""
        summary_parts = []
        if self.current_agent:
            summary_parts.append(f"Current agent: {self.current_agent}")
        if self.previous_agents:
            summary_parts.append(f"Previous agents: {', '.join(self.previous_agents)}")
        
        # Get recent conversation
        recent_msgs = self.get_recent_messages(5)
        if recent_msgs:
            conv_summary = "\nRecent conversation:\n"
            for msg in recent_msgs:
                agent_info = f" [{msg.agent_id}]" if msg.agent_id else ""
                conv_summary += f"{msg.role}{agent_info}: {msg.content[:100]}...\n"
            summary_parts.append(conv_summary)
        
        return "\n".join(summary_parts)


class Memory:
    """Memory component for storing agent interactions and patterns"""
    def __init__(self):
        self.agent_interactions = defaultdict(list)
        self.query_patterns = defaultdict(list)
        self.handoff_history = []
        self.agent_capabilities = {}
    
    def record_interaction(self, agent_id: str, query: str, response: str, success: bool = True):
        """Record an agent interaction"""
        self.agent_interactions[agent_id].append({
            'query': query,
            'response': response,
            'success': success,
            'timestamp': datetime.now()
        })
    
    def record_handoff(self, from_agent: str, to_agent: str, reason: str):
        """Record a handoff event"""
        self.handoff_history.append({
            'from': from_agent,
            'to': to_agent,
            'reason': reason,
            'timestamp': datetime.now()
        })
    
    def update_agent_capability(self, agent_id: str, capability: str, examples: List[str]):
        """Update understanding of agent capabilities"""
        if agent_id not in self.agent_capabilities:
            self.agent_capabilities[agent_id] = {}
        self.agent_capabilities[agent_id][capability] = examples
    
    def get_agent_performance(self, agent_id: str) -> Dict[str, Any]:
        """Get performance metrics for an agent"""
        interactions = self.agent_interactions.get(agent_id, [])
        if not interactions:
            return {'total': 0, 'success_rate': 0}
        
        total = len(interactions)
        successful = sum(1 for i in interactions if i['success'])
        return {
            'total': total,
            'success_rate': successful / total if total > 0 else 0,
            'recent_queries': [i['query'] for i in interactions[-5:]]
        }


class BaseAgent(ABC):
    """Abstract base class for all agents"""
    
    def __init__(self, agent_id: str, name: str, description: str):
        self.agent_id = agent_id
        self.name = name
        self.description = description
        self.memory = Memory()
    
    @abstractmethod
    def can_handle(self, query: str, context: ConversationContext) -> Tuple[bool, float]:
        """
        Determine if this agent can handle the query
        Returns: (can_handle: bool, confidence: float between 0-1)
        """
        pass
    
    @abstractmethod
    def process(self, query: str, context: ConversationContext) -> Tuple[str, HandoffDecision, Optional[str]]:
        """
        Process the user query
        Returns: (response, handoff_decision, suggested_agent_id)
        """
        pass
    
    def get_capabilities(self) -> List[str]:
        """Return a list of capabilities this agent has"""
        return []


class APIAgent(BaseAgent):
    """Agent that communicates through API endpoints"""
    
    def __init__(self, agent_id: str, name: str, description: str, 
                 api_endpoint: str, headers: Optional[Dict] = None):
        super().__init__(agent_id, name, description)
        self.api_endpoint = api_endpoint
        self.headers = headers or {}
    
    def can_handle(self, query: str, context: ConversationContext) -> Tuple[bool, float]:
        """Check if API agent can handle the query"""
        # Call API to check if it can handle
        try:
            response = requests.post(
                f"{self.api_endpoint}/can_handle",
                json={"query": query, "context": context.get_context_summary()},
                headers=self.headers,
                timeout=5
            )
            if response.status_code == 200:
                result = response.json()
                return result.get('can_handle', False), result.get('confidence', 0.0)
        except Exception as e:
            logger.error(f"Error checking API agent {self.agent_id}: {e}")
        
        return False, 0.0
    
    def process(self, query: str, context: ConversationContext) -> Tuple[str, HandoffDecision, Optional[str]]:
        """Process query through API"""
        try:
            response = requests.post(
                f"{self.api_endpoint}/process",
                json={
                    "query": query,
                    "context": context.get_context_summary(),
                    "conversation_history": [
                        {"role": m.role, "content": m.content} 
                        for m in context.get_recent_messages()
                    ]
                },
                headers=self.headers,
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json()
                answer = result.get('response', 'No response from API')
                
                # Determine handoff decision
                if result.get('requires_handoff', False):
                    return answer, HandoffDecision.HANDOFF, result.get('suggested_agent')
                else:
                    return answer, HandoffDecision.CONTINUE, None
            else:
                return f"API error: {response.status_code}", HandoffDecision.RETURN_TO_MAIN, None
                
        except Exception as e:
            logger.error(f"Error processing with API agent {self.agent_id}: {e}")
            return f"Error: {str(e)}", HandoffDecision.RETURN_TO_MAIN, None


class FilterAgent(BaseAgent):
    """Example implementation of a filter agent"""
    
    def __init__(self):
        super().__init__(
            agent_id="filter_agent",
            name="Filter Agent",
            description="Handles data filtering queries and creates filter expressions"
        )
        self.filter_keywords = ['filter', 'search', 'find', 'query', 'where', 'condition', 'criteria']
    
    def can_handle(self, query: str, context: ConversationContext) -> Tuple[bool, float]:
        """Check if query is related to filtering"""
        query_lower = query.lower()
        
        # Check for filter-related keywords
        keyword_matches = sum(1 for keyword in self.filter_keywords if keyword in query_lower)
        
        # Calculate confidence based on keyword matches
        confidence = min(keyword_matches * 0.3, 1.0)
        
        # Boost confidence if previous context was filtering
        if context.current_agent == self.agent_id:
            confidence = min(confidence + 0.3, 1.0)
        
        can_handle = confidence > 0.3
        return can_handle, confidence
    
    def process(self, query: str, context: ConversationContext) -> Tuple[str, HandoffDecision, Optional[str]]:
        """Process filtering query"""
        # Simulate filter processing
        if any(action_word in query.lower() for action_word in ['download', 'export', 'add', 'create']):
            # This seems like an action, not filtering
            return (
                "This query seems to involve actions beyond filtering. Let me transfer you to the action agent.",
                HandoffDecision.HANDOFF,
                "action_agent"
            )
        
        # Process the filter
        response = f"I'll help you create a filter. Based on your query: '{query}', here's the filter expression:\n"
        response += f"FILTER: {self._create_mock_filter(query)}"
        
        return response, HandoffDecision.CONTINUE, None
    
    def _create_mock_filter(self, query: str) -> str:
        """Create a mock filter expression"""
        return f"WHERE text CONTAINS '{query}' AND status = 'active'"
    
    def get_capabilities(self) -> List[str]:
        return ["Create filter expressions", "Search data", "Define query conditions"]


class ActionAgent(BaseAgent):
    """Example implementation of an action agent"""
    
    def __init__(self):
        super().__init__(
            agent_id="action_agent",
            name="Action Agent",
            description="Handles actions like downloading CSV, adding to Marketo/Outreach"
        )
        self.action_keywords = ['download', 'export', 'add', 'create', 'csv', 'marketo', 'outreach']
        self.tools = {
            'download_csv': self._download_csv,
            'add_marketo': self._add_marketo,
            'add_outreach': self._add_outreach
        }
    
    def can_handle(self, query: str, context: ConversationContext) -> Tuple[bool, float]:
        """Check if query is related to actions"""
        query_lower = query.lower()
        
        # Check for action-related keywords
        keyword_matches = sum(1 for keyword in self.action_keywords if keyword in query_lower)
        
        # Calculate confidence
        confidence = min(keyword_matches * 0.35, 1.0)
        
        # Boost confidence if previous context was actions
        if context.current_agent == self.agent_id:
            confidence = min(confidence + 0.2, 1.0)
        
        can_handle = confidence > 0.3
        return can_handle, confidence
    
    def process(self, query: str, context: ConversationContext) -> Tuple[str, HandoffDecision, Optional[str]]:
        """Process action query"""
        query_lower = query.lower()
        
        # Check if this is a filtering query
        if any(filter_word in query_lower for filter_word in ['filter', 'search', 'find']) and \
           not any(action_word in query_lower for action_word in self.action_keywords):
            return (
                "This seems to be a filtering query. Let me transfer you to the filter agent.",
                HandoffDecision.HANDOFF,
                "filter_agent"
            )
        
        # Determine which tool to use
        if 'csv' in query_lower or 'download' in query_lower:
            result = self.tools['download_csv'](query)
        elif 'marketo' in query_lower:
            result = self.tools['add_marketo'](query)
        elif 'outreach' in query_lower:
            result = self.tools['add_outreach'](query)
        else:
            result = "I can help you with downloading CSV files, adding to Marketo, or adding to Outreach. Which action would you like to perform?"
        
        return result, HandoffDecision.CONTINUE, None
    
    def _download_csv(self, query: str) -> str:
        """Mock CSV download"""
        return "I've prepared the CSV file for download. The file 'data_export.csv' contains 1,234 records and is ready for download."
    
    def _add_marketo(self, query: str) -> str:
        """Mock Marketo addition"""
        return "Successfully added the data to Marketo. 156 new leads have been created in your Marketo instance."
    
    def _add_outreach(self, query: str) -> str:
        """Mock Outreach addition"""
        return "Successfully added the prospects to Outreach. 89 new prospects have been added to your sequence."
    
    def get_capabilities(self) -> List[str]:
        return ["Download CSV files", "Add leads to Marketo", "Add prospects to Outreach", "Export data"]


class OrchestratorAgent:
    """Main orchestrator agent that manages handoffs between sub-agents"""
    
    def __init__(self, openai_api_key: str, model: str = "gpt-4o"):
        self.agents: Dict[str, BaseAgent] = {}
        self.memory = Memory()
        self.context = ConversationContext()
        self.openai_client = openai.OpenAI(api_key=openai_api_key)
        self.model = model
        
    def register_agent(self, agent: BaseAgent):
        """Register a new agent with the orchestrator"""
        self.agents[agent.agent_id] = agent
        logger.info(f"Registered agent: {agent.name} ({agent.agent_id})")
    
    def _get_agent_descriptions(self) -> str:
        """Get descriptions of all registered agents"""
        descriptions = []
        for agent_id, agent in self.agents.items():
            desc = f"- {agent.name} ({agent_id}): {agent.description}"
            capabilities = agent.get_capabilities()
            if capabilities:
                desc += f"\n  Capabilities: {', '.join(capabilities)}"
            descriptions.append(desc)
        return "\n".join(descriptions)
    
    def _select_best_agent(self, query: str) -> Tuple[Optional[str], float]:
        """Select the best agent for handling the query"""
        candidates = []
        
        for agent_id, agent in self.agents.items():
            can_handle, confidence = agent.can_handle(query, self.context)
            if can_handle:
                candidates.append((agent_id, confidence))
        
        if not candidates:
            return None, 0.0
        
        # Sort by confidence and return the best
        candidates.sort(key=lambda x: x[1], reverse=True)
        return candidates[0]
    
    def _use_llm_for_routing(self, query: str) -> Optional[str]:
        """Use LLM to determine routing when agent selection is unclear"""
        system_prompt = f"""You are an intelligent routing system. Based on the user query and available agents, 
        determine which agent should handle the request. If no agent is suitable, respond with 'MAIN'.
        
        Available agents:
        {self._get_agent_descriptions()}
        
        Current context:
        {self.context.get_context_summary()}
        
        Respond with ONLY the agent_id or 'MAIN'.
        """
        
        try:
            response = self.openai_client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Route this query: {query}"}
                ],
                temperature=0.3,
                max_tokens=50
            )
            
            suggested_agent = response.choices[0].message.content.strip()
            return suggested_agent if suggested_agent != 'MAIN' else None
            
        except Exception as e:
            logger.error(f"Error using LLM for routing: {e}")
            return None
    
    def _handle_with_main_agent(self, query: str) -> str:
        """Handle query with the main orchestrator agent"""
        system_prompt = """You are a helpful AI assistant. Provide a helpful response to the user's query.
        If the query seems like it should be handled by a specialized agent, suggest that in your response."""
        
        try:
            response = self.openai_client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": query}
                ],
                temperature=0.7,
                max_tokens=500
            )
            
            return response.choices[0].message.content
            
        except Exception as e:
            logger.error(f"Error with main agent: {e}")
            return "I apologize, but I encountered an error processing your request. Please try again."
    
    def process_message(self, user_message: str) -> str:
        """Process a user message and route to appropriate agent"""
        # Add user message to context
        self.context.add_message(Message(role="user", content=user_message))
        
        # If we have a current agent, let them try first
        if self.context.current_agent and self.context.current_agent in self.agents:
            current_agent = self.agents[self.context.current_agent]
            
            # Check if current agent can still handle this
            can_handle, confidence = current_agent.can_handle(user_message, self.context)
            
            if can_handle and confidence > 0.5:
                # Let current agent continue
                response, decision, suggested_agent = current_agent.process(user_message, self.context)
                
                if decision == HandoffDecision.CONTINUE:
                    self.context.add_message(Message(
                        role="assistant",
                        content=response,
                        agent_id=current_agent.agent_id
                    ))
                    self.memory.record_interaction(current_agent.agent_id, user_message, response, True)
                    return response
                
                elif decision == HandoffDecision.HANDOFF and suggested_agent:
                    # Handoff to suggested agent
                    self.memory.record_handoff(current_agent.agent_id, suggested_agent, "Agent requested handoff")
                    self.context.previous_agents.append(current_agent.agent_id)
                    self.context.current_agent = suggested_agent
                    
                    if suggested_agent in self.agents:
                        new_agent = self.agents[suggested_agent]
                        handoff_msg = f"[Handoff from {current_agent.name} to {new_agent.name}]\n"
                        new_response, _, _ = new_agent.process(user_message, self.context)
                        
                        full_response = handoff_msg + new_response
                        self.context.add_message(Message(
                            role="assistant",
                            content=full_response,
                            agent_id=suggested_agent
                        ))
                        return full_response
        
        # No current agent or current agent can't handle - find best agent
        best_agent_id, confidence = self._select_best_agent(user_message)
        
        # If no agent has high confidence, use LLM for routing
        if not best_agent_id or confidence < 0.5:
            llm_suggestion = self._use_llm_for_routing(user_message)
            if llm_suggestion and llm_suggestion in self.agents:
                best_agent_id = llm_suggestion
        
        # Route to selected agent or handle with main agent
        if best_agent_id and best_agent_id in self.agents:
            selected_agent = self.agents[best_agent_id]
            
            # Update context
            if self.context.current_agent:
                self.context.previous_agents.append(self.context.current_agent)
            self.context.current_agent = best_agent_id
            
            # Process with selected agent
            response, decision, _ = selected_agent.process(user_message, self.context)
            
            # Add handoff message if switching agents
            if self.context.previous_agents:
                prev_agent_name = self.agents.get(self.context.previous_agents[-1], {}).name
                response = f"[Switching from {prev_agent_name} to {selected_agent.name}]\n{response}"
            
            self.context.add_message(Message(
                role="assistant",
                content=response,
                agent_id=best_agent_id
            ))
            self.memory.record_interaction(best_agent_id, user_message, response, True)
            
            return response
        
        else:
            # No suitable agent found - handle with main orchestrator
            self.context.current_agent = None
            response = self._handle_with_main_agent(user_message)
            
            self.context.add_message(Message(
                role="assistant",
                content=response,
                agent_id="main"
            ))
            
            return response
    
    def get_conversation_history(self) -> List[Dict[str, Any]]:
        """Get the conversation history"""
        return [
            {
                "role": msg.role,
                "content": msg.content,
                "agent": msg.agent_id,
                "timestamp": msg.timestamp.isoformat()
            }
            for msg in self.context.messages
        ]
    
    def reset_conversation(self):
        """Reset the conversation context"""
        self.context = ConversationContext()
        logger.info("Conversation context reset")


# Example usage and testing
def test_orchestrator_system():
    """Test the orchestrator system with various scenarios"""
    
    # Initialize orchestrator (replace with your actual API key)
    orchestrator = OrchestratorAgent(openai_api_key=None)
    
    # Register the example agents
    filter_agent = FilterAgent()
    action_agent = ActionAgent()
    
    orchestrator.register_agent(filter_agent)
    orchestrator.register_agent(action_agent)
    
    # Example: Register API-based agents
    # api_filter_agent = APIAgent(
    #     agent_id="api_filter_agent",
    #     name="API Filter Agent",
    #     description="Filter agent accessed via API",
    #     api_endpoint="http://localhost:8000/filter-agent",
    #     headers={"Authorization": "Bearer your-token"}
    # )
    # orchestrator.register_agent(api_filter_agent)
    
    # Test scenarios
    test_queries = [
        # Filter agent queries
        "I need to filter customers by location",
        "Can you help me search for users created last month?",
        
        # Action agent queries
        "Download the filtered results as CSV",
        "Add these leads to Marketo",
        
        # Handoff scenarios
        "Filter by email domain and then export to CSV",
        "I found the data, now add it to Outreach",
        
        # Main agent fallback
        "What's the weather today?",
        "Tell me a joke",
        
        # Complex routing
        "I need to find active users and download their data",
    ]
    
    print("=" * 80)
    print("Testing Multi-Agent Orchestrator System")
    print("=" * 80)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n--- Test {i}: {query} ---")
        response = orchestrator.process_message(query)
        print(f"Response: {response}")
        print(f"Current Agent: {orchestrator.context.current_agent}")
        print("-" * 40)
    
    # Print conversation history
    print("\n\n=== CONVERSATION HISTORY ===")
    for entry in orchestrator.get_conversation_history():
        print(f"{entry['timestamp']} [{entry['agent']}] {entry['role']}: {entry['content'][:100]}...")


if __name__ == "__main__":
    # Run the test
    test_orchestrator_system()

INFO:__main__:Registered agent: Filter Agent (filter_agent)
INFO:__main__:Registered agent: Action Agent (action_agent)


Testing Multi-Agent Orchestrator System

--- Test 1: I need to filter customers by location ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: I'll help you create a filter. Based on your query: 'I need to filter customers by location', here's the filter expression:
FILTER: WHERE text CONTAINS 'I need to filter customers by location' AND status = 'active'
Current Agent: filter_agent
----------------------------------------

--- Test 2: Can you help me search for users created last month? ---
Response: [Handoff from Filter Agent to Action Agent]
I can help you with downloading CSV files, adding to Marketo, or adding to Outreach. Which action would you like to perform?
Current Agent: action_agent
----------------------------------------

--- Test 3: Download the filtered results as CSV ---
Response: I've prepared the CSV file for download. The file 'data_export.csv' contains 1,234 records and is ready for download.
Current Agent: action_agent
----------------------------------------

--- Test 4: Add these leads to Marketo ---
Response: Successfully added the data to Marketo. 156 new leads have been created in your Mar

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: I don't have real-time capabilities to check the current weather. I recommend using a weather app or website like Weather.com or a voice assistant like Siri or Google Assistant to get the most accurate and up-to-date weather information for your area.
Current Agent: None
----------------------------------------

--- Test 8: Tell me a joke ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: Sure, here's one for you:

Why don't scientists trust atoms?

Because they make up everything!
Current Agent: None
----------------------------------------

--- Test 9: I need to find active users and download their data ---


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response: [Switching from Filter Agent to Action Agent]
I've prepared the CSV file for download. The file 'data_export.csv' contains 1,234 records and is ready for download.
Current Agent: action_agent
----------------------------------------


=== CONVERSATION HISTORY ===
2025-06-05T15:11:06.124888 [None] user: I need to filter customers by location...
2025-06-05T15:11:06.859525 [filter_agent] assistant: I'll help you create a filter. Based on your query: 'I need to filter customers by location', here's...
2025-06-05T15:11:06.859657 [None] user: Can you help me search for users created last month?...
2025-06-05T15:11:06.859685 [action_agent] assistant: [Handoff from Filter Agent to Action Agent]
I can help you with downloading CSV files, adding to Mar...
2025-06-05T15:11:06.859716 [None] user: Download the filtered results as CSV...
2025-06-05T15:11:06.859724 [action_agent] assistant: I've prepared the CSV file for download. The file 'data_export.csv' contains 1,234 records and is re.

# Testing mjlti aganet system 

In [5]:
import openai
import json
import uuid
import requests
import time
from datetime import datetime
from typing import Dict, List, Optional, Any, Tuple
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from enum import Enum
import asyncio
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class AgentType(Enum):
    MAIN = "main"
    API_AGENT = "api_agent"

class HandoffDecision(Enum):
    CONTINUE = "continue"
    HANDOFF_TO_MAIN = "handoff_to_main"
    HANDOFF_TO_AGENT = "handoff_to_agent"
    DIRECT_RESPONSE = "direct_response"

@dataclass
class Message:
    content: str
    role: str
    timestamp: datetime = field(default_factory=datetime.now)
    agent_id: Optional[str] = None
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass
class AgentMemory:
    agent_interactions: Dict[str, List[Message]] = field(default_factory=dict)
    agent_capabilities: Dict[str, List[str]] = field(default_factory=dict)
    context_history: List[Dict[str, Any]] = field(default_factory=list)
    current_agent: Optional[str] = None
    handoff_count: int = 0

class BaseAgent(ABC):
    def __init__(self, agent_id: str, name: str, description: str, capabilities: List[str]):
        self.agent_id = agent_id
        self.name = name
        self.description = description
        self.capabilities = capabilities
        
    @abstractmethod
    def process_query(self, query: str, context: List[Message]) -> Dict[str, Any]:
        pass
    
    @abstractmethod
    def should_handoff(self, query: str, context: List[Message]) -> Tuple[HandoffDecision, Optional[str]]:
        pass

class MainAgent(BaseAgent):
    def __init__(self, openai_api_key: str):
        super().__init__(
            agent_id="main_agent",
            name="Main Orchestrator Agent",
            description="Main agent responsible for routing queries to Filter Agent or Action Agent",
            capabilities=["routing", "orchestration", "context_management", "handoff_decision"]
        )
        self.client = openai.OpenAI(api_key=openai_api_key)
        
    def process_query(self, query: str, context: List[Message]) -> Dict[str, Any]:
        """Process query and decide on agent routing"""
        
        system_prompt = f"""
        You are the Main Orchestrator Agent in a multi-agent system with two specialized agents:
        
        1. FILTER AGENT (filter_agent):
           - Capabilities: data filtering, query processing, search operations
           - Use for: "filter", "search", "find", "show me", "get data", "retrieve"
           - Example queries: "Filter customers by location", "Find all leads from last month"
        
        2. ACTION AGENT (action_agent):
           - Capabilities: add_outreach, get_csv, add_marketo, data actions, automation
           - Use for: "add", "create", "execute", "perform", "automate", "export"
           - Example queries: "Add outreach campaign", "Export to CSV", "Add to Marketo"
        
        Available agents and their capabilities:
        {self._get_available_agents_info()}
        
        Current context: {len(context)} previous messages
        
        Analyze the user query and determine:
        - Does it involve FILTERING/SEARCHING data? → route to filter_agent
        - Does it involve ACTIONS/OPERATIONS? → route to action_agent
        - Is it general conversation? → handle yourself
        
        Provide a JSON response with:
        - "target_agent": agent_id to route to (filter_agent, action_agent, or main_agent)
        - "reasoning": explanation of routing decision
        - "context_summary": brief summary of relevant context
        - "response": your response if handling the query yourself
        - "confidence": confidence level (0-1) in routing decision
        """
        
        # Prepare context for LLM
        context_messages = []
        for msg in context[-10:]:
            context_messages.append({
                "role": msg.role,
                "content": f"[{msg.agent_id}] {msg.content}" if msg.agent_id else msg.content
            })
        
        messages = [
            {"role": "system", "content": system_prompt},
            *context_messages,
            {"role": "user", "content": query}
        ]
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.3,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            return result
            
        except Exception as e:
            logger.error(f"Main agent processing error: {str(e)}")
            return {
                "target_agent": "main_agent",
                "reasoning": f"Error in processing: {str(e)}",
                "context_summary": "Error occurred",
                "response": "I encountered an error processing your request. Please try again.",
                "confidence": 0.0
            }
    
    def should_handoff(self, query: str, context: List[Message]) -> Tuple[HandoffDecision, Optional[str]]:
        """Main agent always processes queries first"""
        return HandoffDecision.CONTINUE, None
    
    def _get_available_agents_info(self) -> str:
        """Get information about available agents"""
        return "This will be populated by the system with available agents"

class APIAgent(BaseAgent):
    def __init__(self, agent_id: str, name: str, description: str, capabilities: List[str], 
                 api_endpoint: str, api_headers: Dict[str, str] = None):
        super().__init__(agent_id, name, description, capabilities)
        self.api_endpoint = api_endpoint
        self.api_headers = api_headers or {"Content-Type": "application/json"}
        self.request_timeout = 30
        self.max_retries = 3
        
    def process_query(self, query: str, context: List[Message]) -> Dict[str, Any]:
        """Process query through external API"""
        
        # Prepare context for API call
        context_data = []
        for msg in context[-5:]:  # Last 5 messages for context
            context_data.append({
                "role": msg.role,
                "content": msg.content,
                "agent_id": msg.agent_id,
                "timestamp": msg.timestamp.isoformat()
            })
        
        payload = {
            "query": query,
            "context": context_data,
            "agent_id": self.agent_id,
            "timestamp": datetime.now().isoformat()
        }
        
        # Make API call with retries
        for attempt in range(self.max_retries):
            try:
                logger.info(f"🌐 Calling {self.name} API (attempt {attempt + 1})")
                response = requests.post(
                    self.api_endpoint,
                    json=payload,
                    headers=self.api_headers,
                    timeout=self.request_timeout
                )
                
                if response.status_code == 200:
                    api_result = response.json()
                    
                    # Analyze response to determine if handoff is needed
                    should_handoff = self._analyze_for_handoff(query, api_result)
                    
                    return {
                        "response": api_result.get("response", "API response received"),
                        "confidence": api_result.get("confidence", 0.8),
                        "should_handoff": should_handoff,
                        "handoff_reason": api_result.get("handoff_reason"),
                        "api_status": "success",
                        "api_metadata": api_result.get("metadata", {}),
                        "tools_used": api_result.get("tools_used", []),
                        "execution_time": api_result.get("execution_time", 0)
                    }
                
                elif response.status_code == 422:
                    # Validation error - likely need different agent
                    logger.warning(f"API validation error: {response.text}")
                    return {
                        "response": "This query seems to be outside my capabilities. Let me handoff to a more suitable agent.",
                        "confidence": 0.1,
                        "should_handoff": True,
                        "handoff_reason": f"API validation error: {response.status_code}",
                        "api_status": "validation_error"
                    }
                
                elif response.status_code >= 500:
                    # Server error - retry
                    logger.warning(f"API server error {response.status_code}, retrying...")
                    time.sleep(2 ** attempt)  # Exponential backoff
                    continue
                
                else:
                    # Client error - probably wrong agent
                    logger.warning(f"API client error: {response.status_code}")
                    return {
                        "response": "I'm not able to handle this type of request. Let me find a better agent for you.",
                        "confidence": 0.2,
                        "should_handoff": True,
                        "handoff_reason": f"API error: {response.status_code}",
                        "api_status": "client_error"
                    }
                    
            except requests.exceptions.Timeout:
                logger.warning(f"API timeout (attempt {attempt + 1})")
                if attempt == self.max_retries - 1:
                    return {
                        "response": "The request timed out. Please try again or rephrase your query.",
                        "confidence": 0.0,
                        "should_handoff": True,
                        "handoff_reason": "API timeout",
                        "api_status": "timeout"
                    }
                time.sleep(2 ** attempt)
                
            except requests.exceptions.ConnectionError:
                logger.error(f"API connection error (attempt {attempt + 1})")
                if attempt == self.max_retries - 1:
                    return {
                        "response": "Unable to connect to the service. Please try again later.",
                        "confidence": 0.0,
                        "should_handoff": True,
                        "handoff_reason": "API connection error",
                        "api_status": "connection_error"
                    }
                time.sleep(2 ** attempt)
                
            except Exception as e:
                logger.error(f"Unexpected API error: {str(e)}")
                return {
                    "response": f"An unexpected error occurred: {str(e)}",
                    "confidence": 0.0,
                    "should_handoff": True,
                    "handoff_reason": f"Unexpected error: {str(e)}",
                    "api_status": "error"
                }
        
        # All retries failed
        return {
            "response": "Service is currently unavailable. Please try again later.",
            "confidence": 0.0,
            "should_handoff": True,
            "handoff_reason": "API service unavailable",
            "api_status": "unavailable"
        }
    
    def should_handoff(self, query: str, context: List[Message]) -> Tuple[HandoffDecision, Optional[str]]:
        """Determine if API agent should handoff"""
        result = self.process_query(query, context)
        
        if result.get("should_handoff", False):
            return HandoffDecision.HANDOFF_TO_MAIN, "main_agent"
        elif result.get("confidence", 0) < 0.3:
            return HandoffDecision.HANDOFF_TO_MAIN, "main_agent"
        elif result.get("api_status") in ["validation_error", "client_error"]:
            return HandoffDecision.HANDOFF_TO_MAIN, "main_agent"
        else:
            return HandoffDecision.DIRECT_RESPONSE, None
    
    def _analyze_for_handoff(self, query: str, api_result: Dict[str, Any]) -> bool:
        """Analyze API response to determine if handoff is needed"""
        
        # Check explicit handoff indicators from API
        if api_result.get("should_handoff", False):
            return True
        
        # Check confidence level
        if api_result.get("confidence", 1.0) < 0.3:
            return True
        
        # Check for error indicators in response
        response = api_result.get("response", "").lower()
        handoff_indicators = [
            "not my expertise", "can't help", "unable to", "outside my capabilities",
            "don't understand", "unclear", "not sure", "different agent"
        ]
        
        for indicator in handoff_indicators:
            if indicator in response:
                return True
        
        # Check if API returned minimal or error response
        if len(response) < 20 and any(word in response for word in ["error", "fail", "unable"]):
            return True
        
        return False

# ====================
# MOCK API SERVERS FOR TESTING
# ====================

class MockAPIServer:
    """Mock API server to simulate your existing agents"""
    
    @staticmethod
    def create_filter_agent_response(query: str, context: List[Dict]) -> Dict[str, Any]:
        """Simulate Filter Agent API response"""
        
        query_lower = query.lower()
        
        # Successful filter operations
        if any(word in query_lower for word in ["filter", "find", "search", "show", "get", "retrieve"]):
            if "timeout" in query_lower:
                # Simulate timeout scenario
                time.sleep(35)
            
            return {
                "response": f"Successfully filtered data based on your query: '{query}'. Found 23 matching records.",
                "confidence": 0.9,
                "should_handoff": False,
                "metadata": {
                    "records_found": 23,
                    "filter_criteria": query,
                    "execution_time": 1.2
                },
                "tools_used": ["data_filter", "query_processor"]
            }
        
        # Edge case: Action-related queries (should handoff)
        elif any(word in query_lower for word in ["add", "create", "export", "marketo", "outreach"]):
            return {
                "response": "This appears to be an action request. I specialize in filtering and searching data.",
                "confidence": 0.2,
                "should_handoff": True,
                "handoff_reason": "Query requires action capabilities, not filtering"
            }
        
        # Edge case: Unclear queries
        elif len(query.strip()) < 5 or query_lower in ["hello", "hi", "test"]:
            return {
                "response": "I need more specific information about what data you want to filter or search for.",
                "confidence": 0.3,
                "should_handoff": True,
                "handoff_reason": "Query too vague for filtering operations"
            }
        
        # Edge case: Complex queries outside scope
        else:
            return {
                "response": "I'm not sure how to filter data based on this query. Could you be more specific?",
                "confidence": 0.4,
                "should_handoff": True,
                "handoff_reason": "Query unclear or outside filtering scope"
            }
    
    @staticmethod
    def create_action_agent_response(query: str, context: List[Dict]) -> Dict[str, Any]:
        """Simulate Action Agent API response"""
        
        query_lower = query.lower()
        
        # Successful action operations
        if "add_outreach" in query_lower or "outreach" in query_lower:
            return {
                "response": "Successfully created outreach campaign with the specified parameters.",
                "confidence": 0.95,
                "should_handoff": False,
                "metadata": {
                    "campaign_id": "ORC_12345",
                    "recipients": 150,
                    "execution_time": 2.1
                },
                "tools_used": ["add_outreach", "campaign_manager"]
            }
        
        elif "get_csv" in query_lower or "export" in query_lower or "csv" in query_lower:
            return {
                "response": "CSV export completed successfully. Download link: https://example.com/export_12345.csv",
                "confidence": 0.92,
                "should_handoff": False,
                "metadata": {
                    "file_size": "2.3MB",
                    "rows_exported": 1247,
                    "execution_time": 3.5
                },
                "tools_used": ["get_csv", "data_exporter"]
            }
        
        elif "add_marketo" in query_lower or "marketo" in query_lower:
            return {
                "response": "Successfully added contacts to Marketo with the specified tags and segmentation.",
                "confidence": 0.88,
                "should_handoff": False,
                "metadata": {
                    "contacts_added": 89,
                    "marketo_program_id": "MKT_9876",
                    "execution_time": 4.2
                },
                "tools_used": ["add_marketo", "marketo_connector"]
            }
        
        # Edge case: Filter-related queries (should handoff)
        elif any(word in query_lower for word in ["filter", "find", "search", "show me"]):
            return {
                "response": "This looks like a filtering request. I specialize in performing actions and operations.",
                "confidence": 0.25,
                "should_handoff": True,
                "handoff_reason": "Query requires filtering capabilities, not actions"
            }
        
        # Edge case: API errors
        elif "error" in query_lower:
            return {
                "response": "Internal service error occurred",
                "confidence": 0.1,
                "should_handoff": True,
                "handoff_reason": "Service error",
                "error": True
            }
        
        # Edge case: Unsupported actions
        else:
            return {
                "response": "I can perform actions like add_outreach, get_csv, and add_marketo. This request doesn't match my available actions.",
                "confidence": 0.3,
                "should_handoff": True,
                "handoff_reason": "Unsupported action type"
            }

# Mock Flask-like API endpoint simulation
def mock_filter_agent_api(request_data):
    """Mock Filter Agent API endpoint"""
    query = request_data.get("query", "")
    context = request_data.get("context", [])
    
    # Simulate various response scenarios
    if "500_error" in query:
        return {"status": 500, "error": "Internal server error"}
    elif "422_error" in query:
        return {"status": 422, "error": "Validation error"}
    elif "timeout" in query:
        time.sleep(35)  # Simulate timeout
        return {"status": 200, "data": MockAPIServer.create_filter_agent_response(query, context)}
    else:
        return {"status": 200, "data": MockAPIServer.create_filter_agent_response(query, context)}

def mock_action_agent_api(request_data):
    """Mock Action Agent API endpoint"""
    query = request_data.get("query", "")
    context = request_data.get("context", [])
    
    # Simulate various response scenarios
    if "500_error" in query:
        return {"status": 500, "error": "Internal server error"}
    elif "422_error" in query:
        return {"status": 422, "error": "Validation error"}
    elif "connection_error" in query:
        raise requests.exceptions.ConnectionError("Connection failed")
    else:
        return {"status": 200, "data": MockAPIServer.create_action_agent_response(query, context)}

# ====================
# ENHANCED API AGENT WITH MOCK INTEGRATION
# ====================

class MockAPIAgent(APIAgent):
    """API Agent that uses mock responses for testing"""
    
    def __init__(self, agent_id: str, name: str, description: str, capabilities: List[str], 
                 mock_api_function):
        # Use a dummy endpoint since we're mocking
        super().__init__(agent_id, name, description, capabilities, "http://mock-api")
        self.mock_api_function = mock_api_function
    
    def process_query(self, query: str, context: List[Message]) -> Dict[str, Any]:
        """Process query through mock API"""
        
        # Prepare context for API call
        context_data = []
        for msg in context[-5:]:
            context_data.append({
                "role": msg.role,
                "content": msg.content,
                "agent_id": msg.agent_id,
                "timestamp": msg.timestamp.isoformat()
            })
        
        payload = {
            "query": query,
            "context": context_data,
            "agent_id": self.agent_id,
            "timestamp": datetime.now().isoformat()
        }
        
        try:
            logger.info(f"🌐 Calling {self.name} Mock API")
            
            # Handle special error cases
            if "connection_error" in query.lower():
                raise requests.exceptions.ConnectionError("Mock connection error")
            elif "timeout" in query.lower():
                raise requests.exceptions.Timeout("Mock timeout")
            
            # Call mock API
            mock_response = self.mock_api_function(payload)
            
            if mock_response["status"] == 200:
                api_result = mock_response["data"]
                should_handoff = self._analyze_for_handoff(query, api_result)
                
                return {
                    "response": api_result.get("response", "API response received"),
                    "confidence": api_result.get("confidence", 0.8),
                    "should_handoff": should_handoff,
                    "handoff_reason": api_result.get("handoff_reason"),
                    "api_status": "success",
                    "api_metadata": api_result.get("metadata", {}),
                    "tools_used": api_result.get("tools_used", []),
                    "execution_time": api_result.get("execution_time", 0)
                }
            
            elif mock_response["status"] == 422:
                return {
                    "response": "This query seems to be outside my capabilities.",
                    "confidence": 0.1,
                    "should_handoff": True,
                    "handoff_reason": f"API validation error: {mock_response.get('error')}",
                    "api_status": "validation_error"
                }
            
            else:
                return {
                    "response": "Service error occurred. Let me find a better agent for you.",
                    "confidence": 0.2,
                    "should_handoff": True,
                    "handoff_reason": f"API error: {mock_response['status']}",
                    "api_status": "server_error"
                }
                
        except requests.exceptions.Timeout:
            return {
                "response": "The request timed out. Please try again.",
                "confidence": 0.0,
                "should_handoff": True,
                "handoff_reason": "API timeout",
                "api_status": "timeout"
            }
        
        except requests.exceptions.ConnectionError:
            return {
                "response": "Unable to connect to the service.",
                "confidence": 0.0,
                "should_handoff": True,
                "handoff_reason": "API connection error",
                "api_status": "connection_error"
            }
        
        except Exception as e:
            return {
                "response": f"An unexpected error occurred: {str(e)}",
                "confidence": 0.0,
                "should_handoff": True,
                "handoff_reason": f"Unexpected error: {str(e)}",
                "api_status": "error"
            }

# ====================
# MULTI-AGENT SYSTEM
# ====================

class MultiAgentSystem:
    def __init__(self, openai_api_key: str):
        """Initialize the multi-agent system"""
        self.openai_api_key = openai_api_key
        
        self.agents: Dict[str, BaseAgent] = {}
        self.memory = AgentMemory()
        self.session_id = str(uuid.uuid4())
        self.active_agent_id = None
        
        # Initialize main agent
        self.main_agent = MainAgent(openai_api_key)
        self.agents[self.main_agent.agent_id] = self.main_agent
        self.memory.current_agent = self.main_agent.agent_id
        self.active_agent_id = self.main_agent.agent_id
        
        # Update main agent with system reference
        self.main_agent._get_available_agents_info = self._get_agents_info
    
    def add_agent(self, agent: BaseAgent) -> None:
        """Add a specialized agent to the system"""
        self.agents[agent.agent_id] = agent
        self.memory.agent_capabilities[agent.agent_id] = agent.capabilities
    
    def _get_agents_info(self) -> str:
        """Get formatted information about all available agents"""
        info = []
        for agent_id, agent in self.agents.items():
            if agent_id != "main_agent":
                info.append(f"- {agent.name} ({agent_id}): {agent.description}")
                info.append(f"  Capabilities: {', '.join(agent.capabilities)}")
        return "\n".join(info)
    
    def process_user_query(self, query: str) -> Dict[str, Any]:
        """Process user query in continuous chat session with agent handoffs"""
        
        # Add user message to memory
        user_message = Message(content=query, role="user")
        self._add_message_to_memory(user_message)
        
        # Get current context
        context = self._get_context_messages()
        
        # Use the currently active agent
        current_agent_id = self.active_agent_id
        current_agent = self.agents[current_agent_id]
        
        logger.info(f"🤖 Current active agent: {current_agent.name}")
        
        # If we're with a specialized agent, first check if it can handle the query
        if current_agent_id != "main_agent":
            result = current_agent.process_query(query, context)
            handoff_decision, target_agent = current_agent.should_handoff(query, context)
            
            if handoff_decision == HandoffDecision.DIRECT_RESPONSE:
                # Specialized agent can handle it
                response_message = Message(
                    content=result.get("response", "No response provided"),
                    role="assistant",
                    agent_id=current_agent_id
                )
                self._add_message_to_memory(response_message)
                
                return {
                    "response": result.get("response", "No response provided"),
                    "agent_used": current_agent.name,
                    "agent_id": current_agent_id,
                    "handoff_occurred": False,
                    "confidence": result.get("confidence", 1.0),
                    "session_id": self.session_id,
                    "api_status": result.get("api_status"),
                    "tools_used": result.get("tools_used", []),
                    "execution_time": result.get("execution_time", 0)
                }
            
            elif handoff_decision == HandoffDecision.HANDOFF_TO_MAIN:
                logger.info(f"🔄 {current_agent.name} handing off to Main Agent")
                logger.info(f"📝 Reason: {result.get('handoff_reason', 'Query outside expertise')}")
                
                # Switch to main agent
                self.active_agent_id = "main_agent"
                current_agent_id = "main_agent"
                current_agent = self.agents[current_agent_id]
                
                # Add handoff message
                handoff_message = Message(
                    content=f"[HANDOFF] {self.agents[self.memory.current_agent].name} handing off to Main Agent. Reason: {result.get('handoff_reason', 'Query requires different expertise')}",
                    role="system",
                    agent_id="system"
                )
                self._add_message_to_memory(handoff_message)
        
        # Process with main agent
        if current_agent_id == "main_agent":
            result = current_agent.process_query(query, context)
            target_agent_id = result.get("target_agent")
            
            if target_agent_id and target_agent_id != "main_agent" and target_agent_id in self.agents:
                logger.info(f"🎯 Main Agent routing to: {self.agents[target_agent_id].name}")
                logger.info(f"💭 Reasoning: {result.get('reasoning', 'Best suited for this query')}")
                
                # Switch active agent
                self.active_agent_id = target_agent_id
                specialized_agent = self.agents[target_agent_id]
                
                # Add handoff message
                handoff_message = Message(
                    content=f"[HANDOFF] Main Agent routing to {specialized_agent.name}. User query: {query}",
                    role="system",
                    agent_id="system"
                )
                self._add_message_to_memory(handoff_message)
                
                # Get response from specialized agent
                specialized_context = self._get_context_messages()
                specialized_result = specialized_agent.process_query(query, specialized_context)
                
                # Store response
                response_message = Message(
                    content=specialized_result.get("response", "Hello! I'm now handling your request."),
                    role="assistant",
                    agent_id=target_agent_id
                )
                self._add_message_to_memory(response_message)
                self.memory.current_agent = target_agent_id
                
                return {
                    "response": specialized_result.get("response", "Hello! I'm now handling your request."),
                    "agent_used": specialized_agent.name,
                    "agent_id": target_agent_id,
                    "handoff_occurred": True,
                    "previous_agent": "Main Agent",
                    "handoff_reason": result.get("reasoning", "Specialized expertise needed"),
                    "confidence": specialized_result.get("confidence", 1.0),
                    "session_id": self.session_id,
                    "api_status": specialized_result.get("api_status"),
                    "tools_used": specialized_result.get("tools_used", []),
                    "execution_time": specialized_result.get("execution_time", 0)
                }
            
            else:
                # Main agent handles the query itself
                response_message = Message(
                    content=result.get("response", "How can I help you?"),
                    role="assistant",
                    agent_id=current_agent_id
                )
                self._add_message_to_memory(response_message)
                self.memory.current_agent = current_agent_id
                
                return {
                    "response": result.get("response", "How can I help you?"),
                    "agent_used": current_agent.name,
                    "agent_id": current_agent_id,
                    "handoff_occurred": False,
                    "reasoning": result.get("reasoning", ""),
                    "confidence": result.get("confidence", 1.0),
                    "session_id": self.session_id
                }
    
    def _add_message_to_memory(self, message: Message) -> None:
        """Add message to memory"""
        if message.agent_id:
            if message.agent_id not in self.memory.agent_interactions:
                self.memory.agent_interactions[message.agent_id] = []
            self.memory.agent_interactions[message.agent_id].append(message)
        
        # Add to context history
        self.memory.context_history.append({
            "message": message,
            "timestamp": message.timestamp,
            "agent_id": message.agent_id
        })
    
    def _get_context_messages(self) -> List[Message]:
        """Get recent context messages"""
        recent_messages = []
        for entry in self.memory.context_history[-20:]:
            recent_messages.append(entry["message"])
        return recent_messages
    
    def get_memory_summary(self) -> Dict[str, Any]:
        """Get summary of current memory state"""
        return {
            "current_agent": self.memory.current_agent,
            "active_agent": self.active_agent_id,
            "active_agent_name": self.agents[self.active_agent_id].name if self.active_agent_id else None,
            "total_messages": len(self.memory.context_history),
            "agents_used": list(self.memory.agent_interactions.keys()),
            "handoff_count": self.memory.handoff_count,
            "session_id": self.session_id
        }
    
    def get_active_agent_info(self) -> Dict[str, Any]:
        """Get information about currently active agent"""
        if self.active_agent_id and self.active_agent_id in self.agents:
            agent = self.agents[self.active_agent_id]
            return {
                "agent_id": agent.agent_id,
                "name": agent.name,
                "description": agent.description,
                "capabilities": agent.capabilities,
                "is_main_agent": agent.agent_id == "main_agent",
                "agent_type": "api_agent" if isinstance(agent, APIAgent) else "main"
            }
        return None
    
    def clear_memory(self) -> None:
        """Clear memory for new session"""
        self.memory = AgentMemory()
        self.memory.current_agent = "main_agent"
        self.active_agent_id = "main_agent"
        self.session_id = str(uuid.uuid4())

# ====================
# COMPREHENSIVE TEST FRAMEWORK
# ====================

class TestFramework:
    def __init__(self, system: MultiAgentSystem):
        self.system = system
        self.test_results = []
        self.passed_tests = 0
        self.failed_tests = 0
    
    def run_test(self, test_name: str, query: str, expected_agent: str = None, 
                 expected_handoff: bool = None, expected_keywords: List[str] = None) -> Dict[str, Any]:
        """Run a single test case"""
        print(f"\n🧪 TEST: {test_name}")
        print(f"📝 Query: {query}")
        
        try:
            # Record initial state
            initial_agent = self.system.get_active_agent_info()
            
            # Process query
            start_time = time.time()
            result = self.system.process_user_query(query)
            execution_time = time.time() - start_time
            
            # Analyze results
            test_result = {
                "test_name": test_name,
                "query": query,
                "result": result,
                "execution_time": execution_time,
                "initial_agent": initial_agent["name"] if initial_agent else None,
                "final_agent": result["agent_used"],
                "handoff_occurred": result.get("handoff_occurred", False),
                "api_status": result.get("api_status", "N/A"),
                "tools_used": result.get("tools_used", []),
                "confidence": result.get("confidence", 0),
                "passed": True,
                "errors": []
            }
            
            # Validate expectations
            if expected_agent and result["agent_id"] != expected_agent:
                test_result["passed"] = False
                test_result["errors"].append(f"Expected agent '{expected_agent}', got '{result['agent_id']}'")
            
            if expected_handoff is not None and result.get("handoff_occurred", False) != expected_handoff:
                test_result["passed"] = False
                test_result["errors"].append(f"Expected handoff: {expected_handoff}, got: {result.get('handoff_occurred', False)}")
            
            if expected_keywords:
                response_lower = result["response"].lower()
                for keyword in expected_keywords:
                    if keyword.lower() not in response_lower:
                        test_result["passed"] = False
                        test_result["errors"].append(f"Expected keyword '{keyword}' not found in response")
            
            # Display results
            print(f"🏁 Final agent: {result['agent_used']}")
            if result.get("handoff_occurred"):
                print(f"🔄 Handoff: {result.get('previous_agent', 'Unknown')} → {result['agent_used']}")
                print(f"📄 Reason: {result.get('handoff_reason', 'Not specified')}")
            
            print(f"🤖 Response: {result['response'][:200]}{'...' if len(result['response']) > 200 else ''}")
            print(f"📊 Confidence: {result.get('confidence', 0):.2f}")
            print(f"⏱️ Execution time: {execution_time:.2f}s")
            
            if result.get("tools_used"):
                print(f"🛠️ Tools used: {', '.join(result['tools_used'])}")
            
            if test_result["passed"]:
                print("✅ TEST PASSED")
                self.passed_tests += 1
            else:
                print("❌ TEST FAILED")
                for error in test_result["errors"]:
                    print(f"   ❗ {error}")
                self.failed_tests += 1
            
            self.test_results.append(test_result)
            return test_result
            
        except Exception as e:
            print(f"💥 TEST ERROR: {str(e)}")
            test_result = {
                "test_name": test_name,
                "query": query,
                "passed": False,
                "errors": [f"Exception: {str(e)}"],
                "execution_time": 0
            }
            self.test_results.append(test_result)
            self.failed_tests += 1
            return test_result
    
    def run_all_tests(self):
        """Run comprehensive test suite covering all edge cases"""
        
        print("🚀 STARTING COMPREHENSIVE MULTI-AGENT TEST SUITE")
        print("=" * 80)
        
        # ==================
        # 1. NORMAL OPERATION TESTS
        # ==================
        print("\n📋 SECTION 1: NORMAL OPERATION TESTS")
        
        # Filter Agent tests
        self.run_test(
            "Filter Agent - Basic Query",
            "Filter customers by location California",
            expected_agent="filter_agent",
            expected_handoff=True,
            expected_keywords=["filtered", "records"]
        )
        
        self.run_test(
            "Filter Agent - Search Query",
            "Find all leads from last month",
            expected_agent="filter_agent",
            expected_handoff=True,
            expected_keywords=["found", "matching"]
        )
        
        # Action Agent tests
        self.run_test(
            "Action Agent - Add Outreach",
            "Add outreach campaign for new leads",
            expected_agent="action_agent",
            expected_handoff=True,
            expected_keywords=["outreach", "campaign"]
        )
        
        self.run_test(
            "Action Agent - CSV Export",
            "Export customer data to CSV",
            expected_agent="action_agent",
            expected_handoff=True,
            expected_keywords=["csv", "export"]
        )
        
        self.run_test(
            "Action Agent - Marketo Integration",
            "Add contacts to Marketo with segmentation",
            expected_agent="action_agent",
            expected_handoff=True,
            expected_keywords=["marketo", "contacts"]
        )
        
        # ==================
        # 2. HANDOFF TESTS
        # ==================
        print("\n📋 SECTION 2: HANDOFF LOGIC TESTS")
        
        # Start with one agent, switch to another
        self.run_test(
            "Cross-Agent Handoff - Filter to Action",
            "Show me all California customers",  # Should go to Filter Agent
            expected_agent="filter_agent",
            expected_handoff=True
        )
        
        self.run_test(
            "Cross-Agent Handoff - Action Request",
            "Now add them to outreach campaign",  # Should handoff to Action Agent
            expected_agent="action_agent",
            expected_handoff=True
        )
        
        # Agent recognizes wrong type of request
        self.run_test(
            "Wrong Agent Recognition - Filter Agent",
            "Create a new marketing campaign",  # Filter agent should handoff
            expected_agent="action_agent",  # Should end up with Action Agent
            expected_handoff=True
        )
        
        # ==================
        # 3. ERROR HANDLING TESTS
        # ==================
        print("\n📋 SECTION 3: ERROR HANDLING TESTS")
        
        # API timeout
        self.run_test(
            "API Timeout Handling",
            "Filter data with timeout scenario",
            expected_handoff=True  # Should handoff due to timeout
        )
        
        # API server error
        self.run_test(
            "API Server Error - 500",
            "Filter 500_error data",
            expected_handoff=True  # Should handoff due to server error
        )
        
        # API validation error
        self.run_test(
            "API Validation Error - 422",
            "Filter 422_error data",
            expected_handoff=True  # Should handoff due to validation error
        )
        
        # Connection error
        self.run_test(
            "API Connection Error",
            "Add connection_error outreach",
            expected_handoff=True  # Should handoff due to connection error
        )
        
        # ==================
        # 4. EDGE CASE TESTS
        # ==================
        print("\n📋 SECTION 4: EDGE CASE TESTS")
        
        # Ambiguous queries
        self.run_test(
            "Ambiguous Query",
            "help",
            expected_agent="main_agent",  # Main agent should handle
            expected_handoff=False
        )
        
        # Empty/minimal queries
        self.run_test(
            "Minimal Query",
            "hi",
            expected_handoff=True  # Should trigger handoff due to vagueness
        )
        
        # Complex multi-intent query
        self.run_test(
            "Multi-Intent Query",
            "Filter customers by region and then export to CSV and add to Marketo",
            expected_handoff=True  # Should route to appropriate agent
        )
        
        # Nonsensical query
        self.run_test(
            "Nonsensical Query",
            "purple monkey dishwasher algorithm",
            expected_agent="main_agent",  # Should stay with main agent
            expected_handoff=False
        )
        
        # ==================
        # 5. CONVERSATION CONTINUITY TESTS
        # ==================
        print("\n📋 SECTION 5: CONVERSATION CONTINUITY TESTS")
        
        # Context-dependent follow-up
        self.run_test(
            "Context Follow-up",
            "Show more details about the previous results",
            # Should continue with current agent
            expected_handoff=False
        )
        
        # Agent-specific follow-up
        self.run_test(
            "Agent-Specific Follow-up",
            "Can you export that data?",
            expected_agent="action_agent",  # Should handoff to Action Agent
            expected_handoff=True
        )
        
        # ==================
        # 6. PERFORMANCE TESTS
        # ==================
        print("\n📋 SECTION 6: PERFORMANCE TESTS")
        
        # Multiple rapid queries
        rapid_queries = [
            "Filter leads by score",
            "Add to outreach",
            "Export results",
            "Find high-value customers",
            "Create marketo campaign"
        ]
        
        for i, query in enumerate(rapid_queries):
            self.run_test(
                f"Rapid Query {i+1}",
                query,
                expected_handoff=True
            )
        
        # ==================
        # 7. RECOVERY TESTS
        # ==================
        print("\n📋 SECTION 7: RECOVERY TESTS")
        
        # Recovery after error
        self.run_test(
            "Recovery After Error",
            "Filter customers by location",  # Normal query after errors
            expected_agent="filter_agent",
            expected_handoff=True
        )
        
        # Agent switching after multiple handoffs
        self.run_test(
            "Multiple Handoff Recovery",
            "Add outreach for filtered customers",
            expected_agent="action_agent",
            expected_handoff=True
        )
        
        # ==================
        # TEST SUMMARY
        # ==================
        self.print_test_summary()
    
    def print_test_summary(self):
        """Print comprehensive test summary"""
        print("\n" + "=" * 80)
        print("🏁 TEST SUITE COMPLETE - COMPREHENSIVE SUMMARY")
        print("=" * 80)
        
        print(f"📊 OVERALL RESULTS:")
        print(f"   ✅ Passed: {self.passed_tests}")
        print(f"   ❌ Failed: {self.failed_tests}")
        print(f"   📈 Success Rate: {(self.passed_tests / (self.passed_tests + self.failed_tests) * 100):.1f}%")
        
        # Categorize results
        categories = {
            "Normal Operation": [],
            "Handoff Logic": [],
            "Error Handling": [],
            "Edge Cases": [],
            "Conversation Continuity": [],
            "Performance": [],
            "Recovery": []
        }
        
        for result in self.test_results:
            test_name = result["test_name"]
            if "Basic" in test_name or "Search" in test_name or "Add" in test_name or "CSV" in test_name or "Marketo" in test_name:
                categories["Normal Operation"].append(result)
            elif "Handoff" in test_name or "Cross-Agent" in test_name or "Recognition" in test_name:
                categories["Handoff Logic"].append(result)
            elif "Error" in test_name or "Timeout" in test_name or "Connection" in test_name:
                categories["Error Handling"].append(result)
            elif "Ambiguous" in test_name or "Minimal" in test_name or "Multi-Intent" in test_name or "Nonsensical" in test_name:
                categories["Edge Cases"].append(result)
            elif "Context" in test_name or "Follow-up" in test_name:
                categories["Conversation Continuity"].append(result)
            elif "Rapid" in test_name:
                categories["Performance"].append(result)
            elif "Recovery" in test_name:
                categories["Recovery"].append(result)
        
        print(f"\n📋 RESULTS BY CATEGORY:")
        for category, results in categories.items():
            if results:
                passed = sum(1 for r in results if r["passed"])
                total = len(results)
                print(f"   {category}: {passed}/{total} passed ({passed/total*100:.1f}%)")
        
        # Detailed failure analysis
        failed_results = [r for r in self.test_results if not r["passed"]]
        if failed_results:
            print(f"\n❌ FAILED TESTS ANALYSIS:")
            for result in failed_results:
                print(f"   🔸 {result['test_name']}")
                for error in result.get("errors", []):
                    print(f"     ❗ {error}")
        
        # Performance analysis
        execution_times = [r.get("execution_time", 0) for r in self.test_results if "execution_time" in r]
        if execution_times:
            avg_time = sum(execution_times) / len(execution_times)
            max_time = max(execution_times)
            print(f"\n⏱️ PERFORMANCE METRICS:")
            print(f"   Average execution time: {avg_time:.2f}s")
            print(f"   Maximum execution time: {max_time:.2f}s")
        
        # Agent usage analysis
        agent_usage = {}
        for result in self.test_results:
            if "result" in result:
                agent = result["result"].get("agent_id", "unknown")
                agent_usage[agent] = agent_usage.get(agent, 0) + 1
        
        print(f"\n🤖 AGENT USAGE STATISTICS:")
        for agent, count in agent_usage.items():
            percentage = count / len(self.test_results) * 100
            print(f"   {agent}: {count} times ({percentage:.1f}%)")
        
        print(f"\n🎯 RECOMMENDATIONS:")
        if self.failed_tests > 0:
            print(f"   • Review failed test cases and improve error handling")
            print(f"   • Consider adjusting confidence thresholds for handoff decisions")
            print(f"   • Enhance API error recovery mechanisms")
        
        if any(r.get("execution_time", 0) > 5 for r in self.test_results):
            print(f"   • Optimize API response times and timeout handling")
        
        print(f"   • Monitor agent handoff patterns in production")
        print(f"   • Implement additional edge case handling based on test results")

# ====================
# MAIN EXECUTION
# ====================

if __name__ == "__main__":
    # Initialize the system
    print("🔧 Initializing Multi-Agent System with API Agents...")
    system = MultiAgentSystem(openai_api_key=None)
    
    # Create and add Filter Agent (using mock API)
    filter_agent = MockAPIAgent(
        agent_id="filter_agent",
        name="Filter Agent",
        description="Specialized in data filtering, querying, and search operations",
        capabilities=["data_filtering", "search", "query_processing", "data_retrieval"],
        mock_api_function=mock_filter_agent_api
    )
    system.add_agent(filter_agent)
    
    # Create and add Action Agent (using mock API)
    action_agent = MockAPIAgent(
        agent_id="action_agent",
        name="Action Agent", 
        description="Specialized in performing actions like add_outreach, get_csv, add_marketo",
        capabilities=["add_outreach", "get_csv", "add_marketo", "automation", "data_export"],
        mock_api_function=mock_action_agent_api
    )
    system.add_agent(action_agent)
    
    print("✅ System initialized with agents:")
    for agent_id, agent in system.agents.items():
        print(f"   • {agent.name} ({agent_id})")
        if hasattr(agent, 'capabilities'):
            print(f"     Capabilities: {', '.join(agent.capabilities)}")
    
    # Initialize and run comprehensive test framework
    print("\n🧪 Starting Comprehensive Test Framework...")
    test_framework = TestFramework(system)
    test_framework.run_all_tests()
    
    print("\n🎉 All tests completed! Check the summary above for detailed results.")
    
    # Optional: Interactive testing mode
    print("\n" + "="*60)
    print("💬 INTERACTIVE TESTING MODE")
    print("Enter queries to test the system manually (type 'quit' to exit):")
    print("="*60)
    
    while True:
        try:
            user_input = input("\n👤 Your query: ").strip()
            if user_input.lower() in ['quit', 'exit', 'q']:
                break
            
            if user_input:
                print("🔍 Processing...")
                result = system.process_user_query(user_input)
                
                print(f"🤖 Agent: {result['agent_used']}")
                print(f"📝 Response: {result['response']}")
                if result.get('handoff_occurred'):
                    print(f"🔄 Handoff: {result.get('handoff_reason', 'N/A')}")
                if result.get('tools_used'):
                    print(f"🛠️ Tools: {', '.join(result['tools_used'])}")
                print(f"📊 Confidence: {result.get('confidence', 0):.2f}")
                
        except KeyboardInterrupt:
            print("\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {str(e)}")
    
    print("\n🏁 Interactive testing complete!")

INFO:__main__:🤖 Current active agent: Main Orchestrator Agent


🔧 Initializing Multi-Agent System with API Agents...
✅ System initialized with agents:
   • Main Orchestrator Agent (main_agent)
     Capabilities: routing, orchestration, context_management, handoff_decision
   • Filter Agent (filter_agent)
     Capabilities: data_filtering, search, query_processing, data_retrieval
   • Action Agent (action_agent)
     Capabilities: add_outreach, get_csv, add_marketo, automation, data_export

🧪 Starting Comprehensive Test Framework...
🚀 STARTING COMPREHENSIVE MULTI-AGENT TEST SUITE

📋 SECTION 1: NORMAL OPERATION TESTS

🧪 TEST: Filter Agent - Basic Query
📝 Query: Filter customers by location California


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:🎯 Main Agent routing to: Filter Agent
INFO:__main__:💭 Reasoning: The query involves filtering data to identify customers based on a specific location, which falls under the capabilities of the filter_agent.
INFO:__main__:🌐 Calling Filter Agent Mock API
INFO:__main__:🤖 Current active agent: Filter Agent
INFO:__main__:🌐 Calling Filter Agent Mock API
INFO:__main__:🌐 Calling Filter Agent Mock API
INFO:__main__:🤖 Current active agent: Filter Agent
INFO:__main__:🌐 Calling Filter Agent Mock API
INFO:__main__:🌐 Calling Filter Agent Mock API
INFO:__main__:🔄 Filter Agent handing off to Main Agent
INFO:__main__:📝 Reason: Query requires action capabilities, not filtering


🏁 Final agent: Filter Agent
🔄 Handoff: Main Agent → Filter Agent
📄 Reason: The query involves filtering data to identify customers based on a specific location, which falls under the capabilities of the filter_agent.
🤖 Response: Successfully filtered data based on your query: 'Filter customers by location California'. Found 23 matching records.
📊 Confidence: 0.90
⏱️ Execution time: 2.55s
🛠️ Tools used: data_filter, query_processor
✅ TEST PASSED

🧪 TEST: Filter Agent - Search Query
📝 Query: Find all leads from last month
🏁 Final agent: Filter Agent
🤖 Response: Successfully filtered data based on your query: 'Find all leads from last month'. Found 23 matching records.
📊 Confidence: 0.90
⏱️ Execution time: 0.00s
🛠️ Tools used: data_filter, query_processor
❌ TEST FAILED
   ❗ Expected handoff: True, got: False

🧪 TEST: Action Agent - Add Outreach
📝 Query: Add outreach campaign for new leads


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:🎯 Main Agent routing to: Action Agent
INFO:__main__:💭 Reasoning: The user query involves performing an action, specifically adding an outreach campaign, which falls under the capabilities of the action_agent.
INFO:__main__:🌐 Calling Action Agent Mock API
INFO:__main__:🤖 Current active agent: Action Agent
INFO:__main__:🌐 Calling Action Agent Mock API
INFO:__main__:🌐 Calling Action Agent Mock API
INFO:__main__:🤖 Current active agent: Action Agent
INFO:__main__:🌐 Calling Action Agent Mock API
INFO:__main__:🌐 Calling Action Agent Mock API
INFO:__main__:🤖 Current active agent: Action Agent
INFO:__main__:🌐 Calling Action Agent Mock API
INFO:__main__:🌐 Calling Action Agent Mock API
INFO:__main__:🔄 Action Agent handing off to Main Agent
INFO:__main__:📝 Reason: Query requires filtering capabilities, not actions


...

🔍 Processing...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:🎯 Main Agent routing to: Action Agent
INFO:__main__:💭 Reasoning: The user query involves performing an action, specifically adding data to Marketo, which falls under the capabilities of the action_agent.
INFO:__main__:🌐 Calling Action Agent Mock API


🤖 Agent: Action Agent
📝 Response: Successfully added contacts to Marketo with the specified tags and segmentation.
🔄 Handoff: The user query involves performing an action, specifically adding data to Marketo, which falls under the capabilities of the action_agent.
🛠️ Tools: add_marketo, marketo_connector
📊 Confidence: 0.88



👤 Your query:  exit



🏁 Interactive testing complete!
